### Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pytz
import duckdb
import time

import requests as rq
import json
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Keys

In [2]:
# Get demo API key
def get_demo_key():
    f = open("/home/vikas/Documents/CG_demo_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [3]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

### API status

In [4]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [5]:
def get_response(endpoint, headers, params, URL):

    url = "".join((URL, endpoint))
    response = rq.get(url, headers=headers, params=params)

    try:
        data = response.json()
    except ValueError:
        print("Invalid JSON response")
        return None

    if response.status_code != 200:
        print(f"Failed to fetch data, status code {response.status_code}")

    return data

In [6]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : get_demo_key() 
}

use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [7]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

### Create URLs

In [8]:
def get_url(url_type,
            network = "",
            dex = "",
            pool_address = "",
            token_address = ""):

    url_dict = {
        "new_pools": f"/onchain/networks/new_pools",
        "trending_pools": f"/onchain/networks/{network}/pools",
        "top_pools": f"/onchain/networks/{network}/pools",
        "top_pools_dex": f"/onchain/networks/{network}/dexes/{dex}/pools",
        "specific_pool_dex": f"/onchain/networks/{network}/pools/{pool_address}",
        "top_pools_add": f"/onchain/networks/{network}/tokens/{token_address}/pools",
        "token_data": f"/onchain/networks/{network}/tokens/{token_address}",
        "token_info": f"/onchain/networks/{network}/tokens/{token_address}/info"
    }

    return url_dict[url_type]

### Get list of networks

In [9]:
def get_all_networks():    

    networks_list_response = get_response("/onchain/networks",
                                          use_pro, 
                                          "",
                                          PRO_URL)

    return pd.DataFrame(networks_list_response["data"])

In [10]:
df_all_networks = get_all_networks()

In [11]:
df_all_networks[df_all_networks["id"].str.contains("bsc",
                                                   case=False,
                                                   na=False)]

,id,type,attributes
1,bsc,network,"{'name': 'BNB Chain', 'coingecko_asset_platfor..."


### Get top pools for a specific network

In [12]:
def safe_get(d, path, default=None):
    """Safely get a nested dictionary value."""
    for key in path:
        if isinstance(d, dict) and key in d:
            d = d[key]
        else:
            return default
    return d


def collect_response(list_response):    

    response_all = []

    for response in list_response.get("data", []):
        
        all_attributes = response.get("attributes", {})
        rel = response.get("relationships", {})
        
        # Extract nested values safely
        daily_tx = safe_get(all_attributes, ["transactions", "h24"], {})
        base_token_add = safe_get(rel, ["base_token", "data", "id"], "NA")

        # If token_add exists, split it
        token_add = base_token_add.split("_")[1] if base_token_add != "NA" and "_" in base_token_add else "NA"
        
        temp_dict = dict(
            pair = safe_get(all_attributes, ["name"], "NA"),
            dex = safe_get(rel, ["dex", "data", "id"], "NA"),
            network = safe_get(rel, ["network", "data", "id"], "NA"),
            token_add = token_add,
            pool_add = safe_get(all_attributes, ["address"], "NA"),
            fdv_usd = safe_get(all_attributes, ["fdv_usd"], "NA"),
            market_cap_usd = safe_get(all_attributes, ["market_cap_usd"], "NA"),
            daily_volume = safe_get(all_attributes, ["volume_usd", "h24"], "NA"),
            daily_price_change = safe_get(all_attributes, ["price_change_percentage", "h24"], "NA"),
            daily_buys = safe_get(daily_tx, ["buys"], "NA"),
            daily_sells = safe_get(daily_tx, ["sells"], "NA"),
            daily_buyers = safe_get(daily_tx, ["buyers"], "NA"),
            daily_sellers = safe_get(daily_tx, ["sellers"], "NA")
        )
        
        response_all.append(temp_dict)

    return response_all

In [13]:
def get_top_pools_network(network, sort_by_col):

    target_url = get_url("top_pools", network)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [14]:
get_top_pools_network("solana", "market_cap_usd").head(5)

,pair,dex,network,token_add,pool_add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
19,WTFO / SOL,raydium,NA,8C4RygkxmePm9ys1qCcAB46dUCXNQYTaqfxS5mBrpump,FDGTcewAmNmW5ZnhjJ8yCnYgbnZudqSUZ8EH9ufr2BFY,749979.587165125,749979.587165125,35273.6908347134,14.508,39462,25144,26391,19889
11,pippin / SOL,raydium,NA,Dfh5DzRgSvvCFDoYc2ciTkMrbDfRKybA4SoFbPmApump,8WwcNqdZjCY5Pt7AkhupAFknV2txca9sq6YBkGzLbvdt,342290399.382058,342290399.451805,40768772.8772528,4.392,39420,41275,1147,1693
0,USD1 / SOL,raydium-clmm,NA,USD1ttGY1N17NEEHLmELoaybftRBUSErhqYiQzvEmuB,AQAGYQsdU853WAKhXM79CgNdoyhrRwXvYHX6qrDyC1FS,143879489.199252,2742948478.0951,25602033.836583,0.241,87288,77344,13667,13285
16,USDT / SOL,pancakeswap-v3-solana,NA,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,22HUWiJaTNph96KQTKZVy2wg8KzfCems5nyW7E5H5J6w,2742022826.25177,186195418503.931,11947931.3597422,-0.125,34402,36572,3591,3624
1,SOL / USDC,pancakeswap-v3-solana,NA,So11111111111111111111111111111111111111112,DJNtGuBGEQiUCWE8F981M2C3ZghZt2XLD8f2sQdZ6rsZ,1758449719.79441,1758449719.7944,75662547.8470906,-0.188,69378,66800,2711,2615


### Get data for a specific pool address

In [15]:
def collect_pool_response(list_response):    

    response = list_response["data"]
    all_attributes = response["attributes"]
    daily_tx = all_attributes["transactions"]["h24"]
    rel = response["relationships"]
        
    response_dict = dict(
        pair = all_attributes["name"],
        dex = rel["dex"]["data"]["id"],
        add = all_attributes["address"],
        fdv_usd = all_attributes["fdv_usd"],
        market_cap_usd = all_attributes["market_cap_usd"],
        daily_volume = all_attributes["volume_usd"]["h24"],
        daily_price_change = all_attributes["price_change_percentage"]["h24"],
        daily_buys = daily_tx["buys"],
        daily_sells = daily_tx["sells"],
        daily_buyers = daily_tx["buyers"],
        daily_sellers = daily_tx["sellers"]
    )

    return response_dict

In [16]:
def get_pool_data(network, dex, pool_address):

    target_url = get_url("specific_pool_dex", network, dex, pool_address)

    pool_list_response = get_response(target_url,
                                      use_pro, 
                                      "",
                                      PRO_URL)

    pool_all = collect_pool_response(pool_list_response)   

    return pool_all

In [17]:
get_pool_data("solana", "", "FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz")

{'pair': 'PENGU / SOL',
 'dex': 'orca',
 'add': 'FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz',
 'fdv_usd': '973966372.905152',
 'market_cap_usd': '797984898.179389',
 'daily_volume': '463875.414093402',
 'daily_price_change': '-0.06',
 'daily_buys': 944,
 'daily_sells': 973,
 'daily_buyers': 92,
 'daily_sellers': 75}

### Get data for a specific token address

In [18]:
def collect_token_response(list_response):    

    response = list_response.get("data", {})
    all_attributes = response.get("attributes", {})
    
    # Safely extract launchpad_details or default to empty dict
    launchpad_details = all_attributes.get("launchpad_details", {})

    response_dict = dict(
        name = all_attributes.get("name", ""),
        price_usd = all_attributes.get("price_usd", None),
        fdv_usd = all_attributes.get("fdv_usd", None),
        grad_pert = (
            launchpad_details.get("graduation_percentage")
            if launchpad_details else 0
        ),
        completed = launchpad_details.get("completed", False),
        completed_at = launchpad_details.get("completed_at", None),
        dest_pool = launchpad_details.get("migrated_destination_pool_address", None)
        
    )

    return response_dict

In [19]:
def get_token_data(network, token_address):

    target_url = get_url("token_data",
                         network,
                         "",
                         "",
                         token_address)

    token_list_response = get_response(target_url,
                                  use_pro, 
                                  "",
                                  PRO_URL)

    token_all = collect_token_response(token_list_response)   

    return token_all

### Track tokens

In [20]:
list_of_tokens = [
    "96nXL5td1DxpySaYK8yvkazfteyi6YwqVJYx4w1tpump",
    "BdD7EsVWdNbiM5PZJsBEmZj4VdCrC3myCDTHr2G9moon",
    "7JPBqCzTyVBWJEL79jBvRYBReCKqBSFU1HQyo3jHwave",
    "DhSx8SRk9A3sNze2B2Pk9dnDsfcgQZufsqyWm4qkwave",
    "pHMXKasJ6mrEKJAD5sXREVGkpphLJYjN3yRwWVnpump",
    "x8KtZeEW4WHBk8YhtVmNc3ga3B4GSN1gLDTVKe5wave",
    "5SEjkg4TZx2dPriqvbCCMt6GQCUMXWyhLNR7r6BEpump",
    "9d35fGcAktLZbaRAhTxvGssWS1TG3wXt4ZTKDNZvpump"
]

In [21]:
def track_status(list_of_tokens, network, threshold):

    all_token_data = []

    for token in list_of_tokens:
        token_data = get_token_data(network, token)
        token_data["network"] = network
        token_data["token_add"] = token
        all_token_data.append(token_data)

    df_token = pd.DataFrame(all_token_data)
    
    # Fill NaN with 0's   
    df_token["grad_pert"] = df_token["grad_pert"].fillna(0)
    
    # Convert to datetime with UTC timezone
    df_token["completed_at"] = pd.to_datetime(
        df_token["completed_at"],
        utc=True,
        errors='coerce'  # Invalid strings will become NaT
    )

    # Convert to CET (Central European Time)
    df_token["completed_at"] = df_token["completed_at"].dt.tz_convert("Europe/Berlin")
    
    # Identify rows with graduation % above a certain threshold
    df_token["above_threshold"] = df_token["grad_pert"] > threshold

    # Move token column to the end
    column_to_move = 'token_add'

    # Reorder columns
    df_token = (
        df_token[[col for col in df_token.columns
                  if col != column_to_move]
                          + [column_to_move]]
    )

    return df_token

In [22]:
df_status = track_status(list_of_tokens, "solana", 95)

df_status

,name,price_usd,fdv_usd,grad_pert,completed,completed_at,dest_pool,network,above_threshold,token_add
0,ALFRED Robot Butler Token,None,None,0.19,False,NaT,None,solana,False,96nXL5td1DxpySaYK8yvkazfteyi6YwqVJYx4w1tpump
1,MARIO Meme Token,0.000003716007713,3716.0077128251,0.69,False,NaT,None,solana,False,BdD7EsVWdNbiM5PZJsBEmZj4VdCrC3myCDTHr2G9moon
2,Resonance Quantum Coin,None,None,0.07,False,NaT,None,solana,False,7JPBqCzTyVBWJEL79jBvRYBReCKqBSFU1HQyo3jHwave
3,StonksCoin,0.00001201925216,5752.1616209574,27.89,False,NaT,None,solana,False,DhSx8SRk9A3sNze2B2Pk9dnDsfcgQZufsqyWm4qkwave
4,European Innovation,0.000002731998109,2731.9981092044,100.00,True,2025-12-07 17:54:13+01:00,65B2xsKv1w2NdyiPBBy5nDc2V77wZAQkvPw1fwWd9xGX,solana,True,pHMXKasJ6mrEKJAD5sXREVGkpphLJYjN3yRwWVnpump
5,Orca Mascot,None,None,0.00,False,NaT,None,solana,False,x8KtZeEW4WHBk8YhtVmNc3ga3B4GSN1gLDTVKe5wave
6,Bad Luck Brian,0.000004152078188,4152.078188472,100.00,True,2025-12-07 18:41:40+01:00,ALoFpEmdVyBQ7HPatdYPpaEJS5DEdBWYC22QVzMafXMg,solana,True,5SEjkg4TZx2dPriqvbCCMt6GQCUMXWyhLNR7r6BEpump
7,Talabat,0.000004659220819,4659.220819,0.00,False,NaT,None,solana,False,9d35fGcAktLZbaRAhTxvGssWS1TG3wXt4ZTKDNZvpump


### Notification email

In [23]:
# Get sender password
def get_app_password():
    f = open("/home/vikas/Documents/Gmail_app_pass.json")
    key_dict = json.load(f)
    return key_dict["pass"]

In [24]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_graduation_alert(to_email, subject, html_body):
    
    # Replace with your Gmail email address
    sender_email = 'nftdemoapp@gmail.com'
    
    # Replace with your Gmail app password
    sender_password = get_app_password()

    # Email content
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Attach the body of the email
    msg.attach(MIMEText(html_body, 'html'))

    # Establish a connection to the SMTP server
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        
        # Send the email
        server.sendmail(sender_email, to_email, msg.as_string())

In [25]:
def send_email(list_of_tokens, network, threshold):

    df_status = track_status(list_of_tokens, network, threshold)
    
    df_to_send = df_status[df_status.completed == True]   

    html_body = f"""
                <html>
                  <body>
                    <p>Hello,<br><br>
                       Here are the latest graduation alerts:
                    </p>
                    {df_to_send.to_html(index=False, border=1)}
                    <p>Regards,<br>Your Crypto Alert Bot</p>
                  </body>
                </html>
                """

    try:
        send_graduation_alert('vikas.negi10@gmail.com', 'Token Graduation Alert', html_body)
    except Exception as e:
        print(f"Unable to send the alert email: {e}")

In [26]:
send_email(list_of_tokens, "solana", 95)

### Get new pools

In [27]:
def get_new_pools(sort_by_col, num_rows, pages=5):
    
    target_url = get_url("new_pools")
    
    all_results = []

    for page in range(1, pages + 1):
        params = {"page": page}
        
        toppool_list_response = get_response(
            target_url,   
            use_pro,      
            params,       
            PRO_URL       
        )

        # Skip if request failed or returned nothing
        if not toppool_list_response:
            continue

        # Collect_response returns a list of dicts
        toppool_page = collect_response(toppool_list_response)
        all_results.extend(toppool_page)

    # If nothing was collected, return an empty DataFrame.
    if not all_results:
        return pd.DataFrame()

    df = pd.DataFrame(all_results)

    return (
        df.sort_values(by=sort_by_col, ascending=False)
          .head(num_rows)
    )


def get_network_token_add_dict(sort_by_col, num_rows):
    
    df_new_tokens = get_new_pools(sort_by_col, num_rows)
    
    return df_new_tokens.groupby("network").agg(list)["token_add"].to_dict()

### Track tokens from new pools

In [28]:
def track_status_all(sort_by_col, num_rows):
    
    tokens_dict = get_network_token_add_dict(sort_by_col, num_rows)
    
    dfs = []
    
    for network, list_of_tokens in tokens_dict.items():
        print("Analyzing tokens for network:", network)
        df_status = track_status(list_of_tokens, network, 95)
        dfs.append(df_status)
        
    # Merge into one DataFrame
    df_final = pd.concat(dfs, ignore_index=True)
    
    df_final = df_final.drop_duplicates()
    
    return df_final.sort_values(by = ["grad_pert"],
                                ascending = False)

In [32]:
df_all_status = track_status_all("market_cap_usd", 200)

Analyzing tokens for network: base
Analyzing tokens for network: bsc
Analyzing tokens for network: cro
Analyzing tokens for network: optimism
Analyzing tokens for network: solana


### DuckDB database

#### Filter token address and network

In [37]:
df_all_status_filtered = df_all_status[
                                      (df_all_status.grad_pert < 50) &
                                      (df_all_status.grad_pert > 5)
                                      ]

#### Write token address and network

In [38]:
# Creates the database file if it does not exist
con = duckdb.connect("Token_data.duckdb")

df_to_write = df_all_status_filtered[["network", "token_add"]]

# df_to_write is your pandas DataFrame
con.execute("DROP TABLE IF EXISTS token_add_network")
con.execute("CREATE TABLE IF NOT EXISTS token_add_network AS SELECT * FROM df_to_write")

#### Read token address and network

In [39]:
df_token_add_network = con.execute("SELECT * FROM token_add_network").df()

tokens_dict = df_token_add_network.groupby("network").agg(list)["token_add"].to_dict()

#### Write token data

In [40]:
def track_status_all_from_db(tokens_dict):
    
    dfs = []
    
    for network, list_of_tokens in tokens_dict.items():
        print("Analyzing tokens for network:", network)
        df_status = track_status(list_of_tokens, network, 95)
        dfs.append(df_status)
        
    # Merge into one DataFrame
    df_final = pd.concat(dfs, ignore_index=True)
    
    df_final = df_final.drop_duplicates()
    
    # Add timestamp
    df_final["timestamp"] = pd.Timestamp.now()
    
    return df_final.sort_values(by = ["grad_pert"],
                                ascending = False)

#### Update every 5 minutes

In [ ]:
con = duckdb.connect("Token_data.duckdb")

# Remove previous table if exists
con.execute("DROP TABLE IF EXISTS token_status")

df_status_to_write = track_status_all_from_db(tokens_dict)

# Create a new table
con.execute("CREATE TABLE IF NOT EXISTS token_status AS SELECT * FROM df_status_to_write")

while True:
    
    time.sleep(300)
    
    df_status_to_write = track_status_all_from_db(tokens_dict)
    
    con.execute("INSERT INTO token_status SELECT * FROM df_status_to_write")

Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyzing tokens for network: bsc
Analyzing tokens for network: solana
Analyz

In [45]:
con.close()